## References
### Python tutorials:
-    https://docs.python.org/3/
-    https://www.diveinto.org/python3/
-    https://python.swaroopch.com/
-    https://realpython.com/
-    https://greenteapress.com/wp/think-python/
-    https://www.learnpythonthehardway.org/
### Blogs:
-    http://simeonfranklin.com/blog/  [Generators, Decorators]
-    https://jeffknupp.com/
-    http://www.laurentluce.com/
    
    
### People to follow:
-    Raymond Hettinger (twitter: @raymondh)
-    David Beazley (twitter: @dabeaz)
-    Dan Bader (twitter: @dbader_org)
-    Guido Van Rossum (twitter: @gvanrossum)

### YouTube:
-    Raymond Hettinger's talks and videos
-    https://www.youtube.com/playlist?list=PLRVdut2KPAguz3xcd22i_o_onnmDKj3MA
    
-    David Beazley's talks and videos
-    https://www.youtube.com/user/dabeazllc/videos

### Other sites:
-    https://awesome-python.com/
-    http://www.realpython.com/
-    http://pymotw.com/


## Parellelism vs Concurrency 
- Parallelism: Parallel execution
- Concurrency: Time sharing based execution


Good read for basics for concurrency, parallelism, threading etc
- https://realpython.com/python-concurrency/

## Generators
Generator will yield CPU. 
- More info http://book.pythontips.com/en/latest/generators.html

1. ***for*** takes an iterator and calls next for us
2. **iterable protocol** in python, iterables are collection that return an iterator that can be iterated upon using method next().
3. When we put a yield in a function it becomes a generator and responds to iterator protocol. So the python interpretor will decorate it and assume its a proxy function and return a generator instead of executing it. So now we can iterate over it.

In [6]:
from time import sleep

# basic implementation
def fib(n):
    a, b = 0, 1
    for _ in range(n):
        print(a)
        a, b = b, a + b

# better implementation using list but it first generates the list then prints it
def fib_list(n):
    series = [0, 1]
    for _ in range(n-2):
        series.append(series[-1] + series[-2])
        sleep(0.5)
    return series

# Using generator we dont have to wait till the function is executed completely
# to get value we can return value once it is ready
def fib_gen(n):
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b
        sleep(0.5)

if __name__ == '__main__':
    #fib(10)
    #for i in fib_list(10):
    for i in fib_gen(10):
        print(i)

0
1
1
2
3
5
8
13
21
34


## Co-routine
Co-routine, a method that takes input, processes it and then returns output
- More info at http://book.pythontips.com/en/latest/coroutines.html

In [7]:
def total():
    print("In total...")
    t = 0
    while True:
        print("total(): waiting for input...")
        # yeild on RHS means wait for input from user
        v = yield
        print("total(): got", v)
        t += v
        print("total(): yielding out", t)
        # send something to caller
        yield t

In [12]:
g = total()
next(g)
g.send(10)
next(g)

In total...
total(): waiting for input...
total(): got 10
total(): yielding out 10
total(): waiting for input...


## concurrent execution using generators
In below example foo() and bar() will yield CPU after printing a value and hence concurrent execution occurs. After bar() has completed only foo() will be executed

In [13]:
# two_functions.py
from time import sleep
from collections import deque

def foo():
    for i in range(10):
        print("foo: counting", i)
        yield
        sleep(0.5)

def bar():
    for i in range(5):
        print("bar: counting", i)
        yield
        sleep(0.5)

if __name__ == '__main__':
    t1 = foo()
    t2 = bar()
    tasks = deque([t1, t2])

    while tasks:
        t = tasks[0]
        try:
            next(t)
        except StopIteration:
            tasks.popleft()
        else:
            tasks.rotate(-1)

foo: counting 0
bar: counting 0
foo: counting 1
bar: counting 1
foo: counting 2
bar: counting 2
foo: counting 3
bar: counting 3
foo: counting 4
bar: counting 4
foo: counting 5
foo: counting 6
foo: counting 7
foo: counting 8
foo: counting 9


## gevent
Its co-routine based python lib. 
- More-info: http://www.gevent.org/intro.html

1. gevent: lightweight scheduling tasks concurrently
2. import gevent
3. gevent spawn creates a greenlet which is nothing but a light weight thread.
4. gevent should not be used if we are not sure of the run time of thread.
5. If one greenlet blocks infinitely then others will block. 
6. Suitable for I/O intensive tasks not for CPU intensive tasks.
7. Cooperative multitasking, to be used when function is blocking most of the time and its CPU usage is in spikes so that other greenlet can utilize the CPU in that time.

In [14]:
# gevent_simple.py
import gevent
from time import sleep

def foo():
    for i in range(10):
        print("foo: counting", i)
        # sleeptime will decide when the CPU is yielded
        gevent.sleep(0)
        # this will block CPU and thus other routines waiting for CPU will also not execute
        #if i >= 5:
        #    while True:
        #        print("foo running forever...")
        #        sleep(0.5)

def bar():
    for i in range(10):
        print("bar: counting", i)
        gevent.sleep(0)

if __name__ == '__main__':
    t1 = gevent.spawn(foo)
    t2 = gevent.spawn(bar)
    print(t1, t2)
    gevent.joinall([t1, t2])


<Greenlet at 0x2073bc09bf8: foo> <Greenlet at 0x2073bc09e18: bar>
foo: counting 0
bar: counting 0
foo: counting 1
bar: counting 1
foo: counting 2
bar: counting 2
foo: counting 3
bar: counting 3
foo: counting 4
bar: counting 4
foo: counting 5
bar: counting 5
foo: counting 6
bar: counting 6
foo: counting 7
bar: counting 7
foo: counting 8
bar: counting 8
foo: counting 9
bar: counting 9


## Threading in python
- Threading in python operates on 1:1 model that is one OS thread for one userspace thread
- More info: https://realpython.com/intro-to-python-threading/


In [14]:
# two_threads.py
from threading import Thread
from time import sleep
from itertools import count

def foo():
    for i in count():
        print("foo: counting", i)
        #sleep(0.5)

def bar():
    for i in count():
        print("bar: counting", i)
        #sleep(0.5)

if __name__ == '__main__':
    t1 = Thread(target=foo)
    t2 = Thread(target=bar)

    t1.start()
    t2.start()
    for i in range(5):
        print("main: counting", i)
        sleep(0.5)
    print("main: complete...")
    
    # to block the main thread while others till others are done
    t1.join()
    t2.join()


foo: countingbar: countingmain:counting  0
foo: counting 1
foo: counting 2
foo: counting 3
foo: counting 0
bar: counting 1
 4
foo: counting 5
foo: counting 6
foo: counting 7
foo: counting 8
foo: counting 90
main:counting 1
main:counting 2
main:counting 3
main:counting 4

bar: counting 2
bar: counting 3
bar: counting 4
bar: counting 5
bar: counting 6
bar: counting 7
bar: counting 8
bar: counting 9


1. Main context from where the threads are created can exit but the process will not terminate till all the threads have completed execution and exit.
2. Process will exit only when all threads have exited.
3. Threads that keep the process active are called as active threads.
4. There is no concept of parent-child thread in python i.e. main thread can exit before the threads that it have spawned.


## GIL
1. ***GIL: global interpreter lock
2. ***Python is interpreted language. 
3. ***Threads in python don't run parallely. All threads run on the same core.
4. ***Since they have to execute using same interpreter.
5. ***So when a thread gets to run it first acquires this GIL then executes.
6. ***Threads in python are concurrent not parallel

- sys.getswitchinterval() : time after which a thread will be prempted if any other thread is waiting for execution. 
- Python threads were never desgined for parallel execution. They are used for preemptive execution. Because of this they are slower than other language threads.

***If we want to do parallel execution we should use multiprocessing. Where each process has its own VM/interpretor environment***

- pypy: faster and better threading support, runtime improvements, if we want performance then we should switch the runtime to pypy . More info https://pypy.org/
- cython: https://cython.org/
- numba: even better but some numba specific syntax has to be learnt. More info https://numba.pydata.org/.

In [16]:
#gen_prime.py
"""
A simple python program to test CPU performance by
generating a series of prime numbers
"""
from __future__ import print_function
import sys
if sys.version_info[0] == 2:
    from __builtin__ import xrange as range

NUM_PRIMES = 100000

def is_prime(number):
    "Returns True if 'number' is prime"
    limit = int(number ** 0.5) + 1
    for i in range(2, limit):
        if number % i == 0:
            return False
    return True

def gen_prime(num):
    "Generates first 'num' series of prime numbers"
    i = 2
    while num:
        if is_prime(i):
            #print(i)
            num -= 1
        i += 1

from time import time

print("Generating %d prime numbers..." % NUM_PRIMES)
start = time()
gen_prime(NUM_PRIMES)
duration = time() - start
print("gen_prime(%d) took %f seconds" % (NUM_PRIMES, duration))

Generating 100000 prime numbers...
gen_prime(100000) took 8.600100 seconds


In [ ]:
# To run with pypy use pypy to run 
pypy gen_prime.py

In [ ]:
#gen_prime_numba.py
"""
A simple python program to test CPU performance by
generating a series of prime numbers
"""
from __future__ import print_function
from numba import jit, boolean, int32, void
import sys
if sys.version_info[0] == 2:
    from __builtin__ import xrange as range

NUM_PRIMES = 100000

@jit(boolean(int32))
def is_prime(number):
    "Returns True if 'number' is prime"
    limit = int(number ** 0.5) + 1
    for i in range(2, limit):
        if number % i == 0:
            return False
    return True

@jit(void(int32))
def gen_prime(num):
    "Generates first 'num' series of prime numbers"
    i = 2
    while num:
        if is_prime(i):
            print(i)
            num -= 1
        i += 1

from time import time

print("Generating %d prime numbers..." % NUM_PRIMES)
start = time()
gen_prime(NUM_PRIMES)
duration = time() - start
print("gen_prime(%d) took %f seconds" % (NUM_PRIMES, duration))



## Thread group
- Used when we have multiple threads and don't know which one will exit first and we want to wait for threads to complete before exiting.
- Since the order is unknown we will wait on the group. 
- Python does not natively support ThreadGroup yet.
- So we wil have to implement our own. Check below.

In [17]:
#ThreadGroup example

# thread_join.py
from threading import Thread
from time import sleep

def foo():
    for i in range(15):
        print("foo: counting", i)
        sleep(0.5)

def bar():
    for i in range(10):
        print("bar: counting", i)
        sleep(0.5)

from collections import deque

if __name__ == '__main__':
    t1 = Thread(target=foo, name="foo")
    t2 = Thread(target=bar, name="testfn")
    t1.start()
    t2.start()
    print("Created two threads:", t1, t2)
    
    threads = deque([t1, t2])
    while threads:
        t = threads[0]
        # Block for 0.1 second
        t.join(0.1)
        # check if the thread is alive
        if not t.is_alive():
            # If no longer active then remove from deque
            threads.popleft()
            print(t.name, "completed...")
        else:
            # else check if other thread has exited
            threads.rotate(-1)

foo: countingbar: countingCreated two threads: <Thread(foo, started 10072)> <Thread(testfn, started 6964)>
 0
 0
bar: countingfoo: counting 1
 1
bar: countingfoo: counting  2
2
bar: counting 3
foo: counting 3
bar: counting 4
foo: counting 4
bar: counting 5
foo: counting 5
bar: counting 6
foo: counting 6
bar: counting 7
foo: counting 7
bar: counting 8
foo: counting 8
bar: counting 9
foo: counting 9
testfn completed...
foo: counting 10
foo: counting 11
foo: counting 12
foo: counting 13
foo: counting 14
foo completed...


In [ ]:
# --- thread_join_better.py
 from threading import Thread, current_thread
from time import sleep
from random import randrange

def foo(c, s):
    t = current_thread()
    for i in range(c):
        print(f"{t.name}: counting", i)
        sleep(s)

from collections import deque


if __name__ == '__main__':
    threads = deque()
    for i in range(5):
        # starting same thread with different arguments
        t = Thread(target=foo, args=(randrange(5, 30, 5), 0.5))
        t.start()
        threads.append(t)

    while threads:
        t = threads[0]
        t.join(0.1)
        if not t.is_alive():
            threads.popleft()
            print(t.name, "completed...")
        else:
            threads.rotate(-1)

            


- **we can use threading.enumerate() get list of active threads**
- **t.join(val) : val is the amount of time you want the join to block, after that it will unblock and main thread can continue**

In [ ]:
# threading.enumerate() example
from time import sleep

def foo(x):
    from threading import current_thread
    th = current_thread()
    for i in range(x):
        print("foo[{}]: counting {}".format(th.name, i))
        sleep(1)



if __name__ == '__main__':
    import threading

    for i in range(10):
        t = threading.Thread(target=foo, args=(10,))
        t.start()

    for t in threading.enumerate():
        print("Thread {} running".format(t.name))

    for t in threading.enumerate():
        if t.name == 'MainThread': continue
        t.join()
        print("Thread {} completed.".format(t.name))


## ThreadSync: Events
1. Thread termination is not supported. A flag is used to notify the thread when to exit. 
2. Better way of cancelling threads is events
3. Event object is the simplest sync mechanism 

- e = Event()
- e.set() : not a counter its just a state
- e.is_set()
- e.clear() : not a counter its just a state
- e.wait()
- e.wait(val): wait for val seconds then proceed

In [18]:
#Event example
from threading import Thread, Event

from random import random, randint, sample
from time import sleep


data = []

updated = Event()


def update_data():
    while True:
        updated.clear()
        sleep(randint(1, 10))
        data[:] = sample(range(100), 10)
        updated.set()
        sleep(randint(1, 5))


def get_data(name):
    while True:
        print("{} waiting for list update...".format(name), flush=True)
        updated.wait()
        print("In {}: top 5 = {}\n".format(name, str(data[:5])), flush=True)
        print("In {}: bottom 5 = {}\n".format(
            name, str(data[-5:])), flush=True)


update_thread = Thread(target=update_data)

readers = {}

update_thread.start()

for i in range(5):
    readers[i] = Thread(target=get_data, args=("Reader-%d" % i,))
    readers[i].start()


Reader-0 waiting for list update...Reader-1 waiting for list update...
Reader-2 waiting for list update...

Reader-3 waiting for list update...
Reader-4 waiting for list update...
In Reader-1: top 5 = [40, 89, 61, 94, 88]
In Reader-2: top 5 = [40, 89, 61, 94, 88]
In Reader-4: top 5 = [40, 89, 61, 94, 88]

In Reader-4: bottom 5 = [22, 57, 73, 7, 32]

In Reader-0: top 5 = [40, 89, 61, 94, 88]
In Reader-3: top 5 = [40, 89, 61, 94, 88]




Reader-4 waiting for list update...In Reader-1: bottom 5 = [22, 57, 73, 7, 32]
In Reader-0: bottom 5 = [22, 57, 73, 7, 32]

In Reader-3: bottom 5 = [22, 57, 73, 7, 32]



In Reader-2: bottom 5 = [22, 57, 73, 7, 32]
Reader-1 waiting for list update...Reader-0 waiting for list update...

Reader-3 waiting for list update...

In Reader-4: top 5 = [40, 89, 61, 94, 88]

In Reader-0: top 5 = [40, 89, 61, 94, 88]

In Reader-1: top 5 = [40, 89, 61, 94, 88]

Reader-2 waiting for list update...
In Reader-3: top 5 = [40, 89, 61, 94, 88]

In Reader-4: bottom 5 = [22,



In Reader-2: bottom 5 = [22, 57, 73, 7, 32]

In Reader-0: bottom 5 = [22, 57, 73, 7, 32]

In Reader-3: bottom 5 = [22, 57, 73, 7, 32]

Reader-0 waiting for list update...In Reader-1: bottom 5 = [22, 57, 73, 7, 32]
In Reader-4: top 5 = [40, 89, 61, 94, 88]

Reader-2 waiting for list update...


Reader-3 waiting for list update...
In Reader-4: bottom 5 = [22, 57, 73, 7, 32]

In Reader-0: top 5 = [40, 89, 61, 94, 88]
Reader-1 waiting for list update...
In Reader-2: top 5 = [40, 89, 61, 94, 88]

In Reader-3: top 5 = [40, 89, 61, 94, 88]


In Reader-1: top 5 = [40, 89, 61, 94, 88]
In Reader-0: bottom 5 = [22, 57, 73, 7, 32]
Reader-4 waiting for list update...
In Reader-3: bottom 5 = [22, 57, 73, 7, 32]



In Reader-2: bottom 5 = [22, 57, 73, 7, 32]

Reader-0 waiting for list update...Reader-3 waiting for list update...In Reader-4: top 5 = [40, 89, 61, 94, 88]



In Reader-1: bottom 5 = [22, 57, 73, 7, 32]
Reader-2 waiting for list update...

In Reader-4: bottom 5 = [22, 57, 73, 7, 32]
In 


Reader-0 waiting for list update...
Reader-3 waiting for list update...In Reader-1: bottom 5 = [22, 57, 73, 7, 32]


Reader-2 waiting for list update...
In Reader-4: top 5 = [40, 89, 61, 94, 88]

In Reader-0: top 5 = [40, 89, 61, 94, 88]

Reader-1 waiting for list update...
In Reader-3: top 5 = [40, 89, 61, 94, 88]

In Reader-2: top 5 = [40, 89, 61, 94, 88]
In Reader-4: bottom 5 = [22, 57, 73, 7, 32]


In Reader-0: bottom 5 = [22, 57, 73, 7, 32]

In Reader-1: top 5 = [40, 89, 61, 94, 88]
In Reader-3: bottom 5 = [22, 57, 73, 7, 32]


Reader-4 waiting for list update...
In Reader-2: bottom 5 = [22, 57, 73, 7, 32]
Reader-0 waiting for list update...

In Reader-1: bottom 5 = [22, 57, 73, 7, 32]

Reader-3 waiting for list update...
In Reader-4: top 5 = [40, 89, 61, 94, 88]

Reader-2 waiting for list update...In Reader-0: top 5 = [40, 89, 61, 94, 88]


Reader-1 waiting for list update...
In Reader-3: top 5 = [40, 89, 61, 94, 88]

In Reader-4: bottom 5 = [22, 57, 73, 7, 32]

In Reader-2: top



In Reader-3: top 5 = [40, 89, 61, 94, 88]
In Reader-4: bottom 5 = [22, 57, 73, 7, 32]


In Reader-0: bottom 5 = [22, 57, 73, 7, 32]
Reader-2 waiting for list update...Reader-1 waiting for list update...


Reader-0 waiting for list update...Reader-4 waiting for list update...In Reader-1: top 5 = [40, 89, 61, 94, 88]
In Reader-3: bottom 5 = [22, 57, 73, 7, 32]
In Reader-2: top 5 = [40, 89, 61, 94, 88]





Reader-3 waiting for list update...In Reader-2: bottom 5 = [22, 57, 73, 7, 32]
In Reader-0: top 5 = [40, 89, 61, 94, 88]



In Reader-4: top 5 = [40, 89, 61, 94, 88]
In Reader-1: bottom 5 = [22, 57, 73, 7, 32]


Reader-2 waiting for list update...
In Reader-3: top 5 = [40, 89, 61, 94, 88]

Reader-1 waiting for list update...In Reader-3: bottom 5 = [22, 57, 73, 7, 32]
In Reader-0: bottom 5 = [22, 57, 73, 7, 32]
In Reader-4: bottom 5 = [22, 57, 73, 7, 32]




In Reader-2: top 5 = [40, 89, 61, 94, 88]
Reader-3 waiting for list update...
Reader-4 waiting for list update...

Reader-0 wait


In Reader-4: top 5 = [40, 89, 61, 94, 88]
Reader-2 waiting for list update...

Reader-1 waiting for list update...
Reader-0 waiting for list update...Reader-3 waiting for list update...

In Reader-2: top 5 = [40, 89, 61, 94, 88]
In Reader-4: bottom 5 = [22, 57, 73, 7, 32]
In Reader-1: top 5 = [40, 89, 61, 94, 88]



In Reader-3: top 5 = [40, 89, 61, 94, 88]
In Reader-0: top 5 = [40, 89, 61, 94, 88]


In Reader-2: bottom 5 = [22, 57, 73, 7, 32]
In Reader-1: bottom 5 = [22, 57, 73, 7, 32]
Reader-4 waiting for list update...


In Reader-0: bottom 5 = [22, 57, 73, 7, 32]
In Reader-3: bottom 5 = [22, 57, 73, 7, 32]


In Reader-4: top 5 = [40, 89, 61, 94, 88]
Reader-2 waiting for list update...Reader-1 waiting for list update...


Reader-0 waiting for list update...Reader-3 waiting for list update...

In Reader-2: top 5 = [40, 89, 61, 94, 88]
In Reader-4: bottom 5 = [22, 57, 73, 7, 32]
In Reader-1: top 5 = [40, 89, 61, 94, 88]



In Reader-0: top 5 = [40, 89, 61, 94, 88]
In Reader-3: top 5 


Reader-2 waiting for list update...
In Reader-0: top 5 = [40, 89, 61, 94, 88]

In Reader-1: bottom 5 = [22, 57, 73, 7, 32]
Reader-3 waiting for list update...
In Reader-4: bottom 5 = [22, 57, 73, 7, 32]

In Reader-2: top 5 = [40, 89, 61, 94, 88]


In Reader-0: bottom 5 = [22, 57, 73, 7, 32]

Reader-1 waiting for list update...In Reader-3: top 5 = [40, 89, 61, 94, 88]


In Reader-2: bottom 5 = [22, 57, 73, 7, 32]

Reader-4 waiting for list update...Reader-0 waiting for list update...

In Reader-3: bottom 5 = [22, 57, 73, 7, 32]
In Reader-1: top 5 = [40, 89, 61, 94, 88]


In Reader-0: top 5 = [40, 89, 61, 94, 88]

In Reader-4: top 5 = [40, 89, 61, 94, 88]
Reader-2 waiting for list update...

In Reader-1: bottom 5 = [22, 57, 73, 7, 32]
Reader-3 waiting for list update...

In Reader-0: bottom 5 = [22, 57, 73, 7, 32]
In Reader-2: top 5 = [40, 89, 61, 94, 88]

In Reader-4: bottom 5 = [22, 57, 73, 7, 32]


In Reader-3: top 5 = [40, 89, 61, 94, 88]

Reader-1 waiting for list update...
Reader-


Reader-3 waiting for list update...In Reader-4: bottom 5 = [22, 57, 73, 7, 32]

In Reader-2: top 5 = [40, 89, 61, 94, 88]


Reader-0 waiting for list update...In Reader-1: top 5 = [40, 89, 61, 94, 88]


Reader-4 waiting for list update...In Reader-3: top 5 = [40, 89, 61, 94, 88]
In Reader-2: bottom 5 = [22, 57, 73, 7, 32]



In Reader-1: bottom 5 = [22, 57, 73, 7, 32]
In Reader-0: top 5 = [40, 89, 61, 94, 88]


In Reader-3: bottom 5 = [22, 57, 73, 7, 32]
Reader-2 waiting for list update...In Reader-4: top 5 = [40, 89, 61, 94, 88]



In Reader-0: bottom 5 = [22, 57, 73, 7, 32]
Reader-1 waiting for list update...

In Reader-4: bottom 5 = [22, 57, 73, 7, 32]
Reader-3 waiting for list update...

In Reader-2: top 5 = [40, 89, 61, 94, 88]

Reader-0 waiting for list update...In Reader-1: top 5 = [40, 89, 61, 94, 88]


In Reader-3: top 5 = [40, 89, 61, 94, 88]
Reader-4 waiting for list update...In Reader-2: bottom 5 = [22, 57, 73, 7, 32]



In Reader-0: top 5 = [40, 89, 61, 94, 88]
In Reader-


In Reader-1: top 5 = [40, 89, 61, 94, 88]
Reader-4 waiting for list update...

In Reader-3: top 5 = [40, 89, 61, 94, 88]

Reader-2 waiting for list update...
In Reader-0: top 5 = [40, 89, 61, 94, 88]

In Reader-1: bottom 5 = [22, 57, 73, 7, 32]

In Reader-4: top 5 = [40, 89, 61, 94, 88]

In Reader-3: bottom 5 = [22, 57, 73, 7, 32]

In Reader-2: top 5 = [40, 89, 61, 94, 88]
In Reader-0: bottom 5 = [22, 57, 73, 7, 32]


Reader-1 waiting for list update...
In Reader-4: bottom 5 = [22, 57, 73, 7, 32]

Reader-3 waiting for list update...
In Reader-2: bottom 5 = [22, 57, 73, 7, 32]
Reader-0 waiting for list update...

In Reader-1: top 5 = [40, 89, 61, 94, 88]

Reader-4 waiting for list update...In Reader-3: top 5 = [40, 89, 61, 94, 88]


In Reader-0: top 5 = [40, 89, 61, 94, 88]

In Reader-1: bottom 5 = [22, 57, 73, 7, 32]
Reader-2 waiting for list update...In Reader-3: bottom 5 = [22, 57, 73, 7, 32]
In Reader-4: top 5 = [40, 89, 61, 94, 88]




Reader-1 waiting for list update...Reader-3 w

Reader-2 waiting for list update...
In Reader-0: bottom 5 = [22, 57, 73, 7, 32]
In Reader-3: top 5 = [40, 89, 61, 94, 88]
In Reader-4: top 5 = [40, 89, 61, 94, 88]



Reader-1 waiting for list update...
In Reader-2: top 5 = [40, 89, 61, 94, 88]

In Reader-3: bottom 5 = [22, 57, 73, 7, 32]
In Reader-4: bottom 5 = [22, 57, 73, 7, 32]


Reader-0 waiting for list update...
In Reader-1: top 5 = [40, 89, 61, 94, 88]

In Reader-2: bottom 5 = [22, 57, 73, 7, 32]

Reader-4 waiting for list update...Reader-3 waiting for list update...

In Reader-0: top 5 = [40, 89, 61, 94, 88]
In Reader-1: bottom 5 = [22, 57, 73, 7, 32]


Reader-2 waiting for list update...
In Reader-4: top 5 = [40, 89, 61, 94, 88]
In Reader-3: top 5 = [40, 89, 61, 94, 88]


Reader-1 waiting for list update...In Reader-0: bottom 5 = [22, 57, 73, 7, 32]


In Reader-2: top 5 = [40, 89, 61, 94, 88]

In Reader-3: bottom 5 = [22, 57, 73, 7, 32]
In Reader-4: bottom 5 = [22, 57, 73, 7, 32]


In Reader-1: top 5 = [40, 89, 61, 94, 88]
Re

## Background daemon threads
1. This thread will exit/terminate if no other threads are active.
2. Useful for monitor threads.
3. They should not share resources with active thread to avoid crash. Eg. no prints in the monitor thread 
4. Mostly used for resource management and cleanup

In [ ]:
# daemon thread example
from threading import Thread, current_thread
from time import sleep


def foo(x):
    t = current_thread()
    for i in range(x):
        print("{}: counting {}".format(t.name, i))
        sleep(1)


def monitor(interval):
    t = current_thread()
    from itertools import count
    for i in count():
        print("{}: monitoring {}".format(t.name, i))
        sleep(interval)


if __name__ == '__main__':
    a = Thread(target=foo, args=(5,))
    b = Thread(target=foo, args=(15,))
    c = Thread(target=monitor,
               name = "MonitorThread",
               args = (0.5,), daemon = True)
               
    #c.daemon = True

    a.start()
    b.start()
    c.start()

    foo(7)

## Method 2 of creating thread
- We can also create a class from Thread class, we just need to implement the ***run()*** method.
- **Be careful while implementing the methods with same name as the Thread methods and also the constructor. If we do so then we have to call base class method**

In [ ]:
#---- threads_oo.py
from threading import Thread
from time import sleep

class CounterThread(Thread):
    
    def __init__(self, c):
        # this method is multiple inheritence safe
        Thread.__init__(self)
        #super().__init__()
        #super(CounterThread, self).__init__()
        
    def run(self):
        for i in range(10):
            print(f"{self.name}: counting", i)
            sleep(0.5)

if __name__ == '__main__':
    c1 = CounterThread(10)
    c2 = CounterThread(7)
    
    c1.start()
    c2.start()

## ThreadSync: Mutex
- **Mutex, should be used only for sync data races it should NOT be used for flow control**
- from threading import Lock
- Dont use ***lock.release() & lock.acquire()*** directly use ***with lock:*** is recommended as it will garauntee release even if exception occurs inside the lock. This is similar to unique lock in c++

In [ ]:
from threading import Lock
lock = Lock()  # Non- reentrant lock, same thread should not acquire this lock again as it will result in deadlock

lock.acquire()
#critical section
lock.release()


# Recommended
try:
    lock.acquire()
    # critical region
    lock.release()
catch: 
    # do something
finally:
    lock.release()
    
# Best 
with lock:
    #critical region

- Lock() is Non-reentrant lock, same thread should not acquire this lock again as it will result in deadlock
- RLock() is reentrant variant of lock. Should avoid using it as it hits performances and we should rethink design **Allows multiple locking by same thread, but you should release it same number of times to avoid issues**

In [ ]:
from threading import RLock as Lock

## ThreadSync: Barriers
Method of flow control synchronization, we add checkpoint to ensure that all threads have reached that point before moving ahead.

In [ ]:
from threading import Thread, current_thread, Barrier
from time import sleep
from random import randrange

def foo(c):
    t = current_thread()
    print(f"{t.name}: started...")
    sleep(c)
    print(f"{t.name}: woke up...")
    # wait here till all the threads have arrived then move ahead
    msg_barrier.wait()
    print(f"{t.name}: continuuing...")

        
# How many threads to wait for
msg_barrier = Barrier(10)

if __name__ == '__main__':
 
    for i in range(10):
        t = Thread(target=foo, args=(randrange(5, 20),))
        t.start()


## ThreadSync: Condition
Condition, another method of flow control sync

- from threading import Condition
- c = Condition()
- usage, with c:
    c.notify()
    c.wait()
    c.wait_for()
    c.notify_all()   # avoid using this if you want to use broadcast condition then use Event instead

In [ ]:
# Producer consumer example using condition

from threading import Thread, Condition, Lock
from random import randint, random
from time import sleep
from collections import deque

class SynchronizedQueue:
    def __init__(self, capacity):
        self.capacity = capacity
        self.queue = deque()
        self.full = Condition()
        self.empty = Condition()
        self.qlock = Lock()
    
    
    def __str__(self):
        with self.qlock:
            s = str(self.queue)
        return s
    
    def put(self, value):
        with self.full:
            if len(self.queue) >= self.capacity:
                self.full.wait()
        
        with self.empty, self.qlock:
            self.queue.append(value)
            self.empty.notify()
    
    def get(self):
        with self.empty:
            if not self.queue:
                self.empty.wait()
                
        with self.full, self.qlock:
            v = self.queue.popleft()
            self.full.notify()
        
        return v

data = SynchronizedQueue(10)

def producer():
    while True:
        v = randint(10, 99)
        print("producer: adding {}, queue = {}".format(v, data))
        data.put(v)
        sleep(random() / 4)
        
def consumer():
    while True:
        v = data.get()
        print("consumer: fetched {}, queue = {}".format(v, data))
        sleep(random())

if __name__ == '__main__':
    #for i in range(4):
    #    Thread(target=producer).start()
    p = Thread(target=producer)
    p.start()

    for i in range(4):
        Thread(target=consumer).start()
        
    #c = Thread(target=consumer)
    #c.start()

## ThreadSync: Semaphore

1. **Semaphore, is used for bandwidth management. That b/w can be resource limit or resource access limit**
2. **Semaphore can be acquired the number of time it was released before it will locked**
3. Semaphore(0) release first semaphore
4. **Semaphore don't have ownership, it can be acquired by one and other can release. This is the difference between Mutex and Semaphore.**
5. Used in couple for consumer-producer problem, for bandwidth limitation
6. Use BoundedSemaphore if limit is to be maintained for number of times it can acquired/released. But use with caution.

In [ ]:
from threading import Semaphore
from threading import BoundedSemaphore as Semaphore

In [ ]:
# Producer consumer using Semaphore
from threading import Thread, Semaphore, Lock
from time import sleep
from random import randint
from collections import deque


class SimpleQueue:
    def __init__(self, size):
        self.queue = deque()
        self.reader = Semaphore(0)
        self.writer = Semaphore(size)
        self.lock = Lock()

    def show(self):
        print(self.queue)

    def put(self, v):

        self.writer.acquire()

        self.queue.append(v)

        self.reader.release()

    def get(self):
        self.reader.acquire()

        v = self.queue.popleft()

        self.writer.release()

        return v


queue = SimpleQueue(10)


def producer():
    while True:
        v = randint(1, 100)
        print("Produced: ", v)
        queue.put(v)
        sleep(v/100.0)


def consumer():
    while True:
        v = queue.get()
        print("Consumed: ", v)
        sleep((v/100.0) + 0.3)


p = Thread(target=producer)
c = Thread(target=consumer)

p.start()
c.start()

while True:
    queue.show()
    sleep(0.5)


## Inbuilt- Queue
1. Has timeouts for methods that can be used for raising exceptions


In [ ]:
from queue import Queue

q = Queue()

q.get()
q.put()

# it has timeout for methods
q.get(timeout=10)
# Or
q.get(block=False)

# wait for a element to be pciked
q.join()

# consumer can use this method to to notify producer who is waiting for this value to be consumed
q.task_done()

In [ ]:
# Queue example
from threading import Thread
import time
from Queue import Queue

class Send(Thread):
    
    def __init__(self, queue):
        Thread.__init__(self)
        self.queue = queue
        self.count = 0
        
    def run(self):
        while True:
            self.count += 1
            self.queue.put(self.count)
            time.sleep(1)

class Recieve(Thread):

    def __init__(self, queue):
        Thread.__init__(self)
        self.queue = queue
        
    def run(self):
        while True:
            print "Count: ", self.queue.get()
            time.sleep(1)

def test():
    data_queue = Queue(10)

    sender = Send(data_queue)
    reciever = Recieve(data_queue)

    sender.start()
    reciever.start()

## Builtin: LifoQueue
1. Stack with blocking capabilities else you can use list

## Builtin: PriorityQueue
1. Push elements in any order but it returns in sorted order
2. Can be used with any python object which can be compared
3. If the object is not comparable wrap it in a tuple with priority value and object i.e. (1, object)

In [1]:
#PriorityQueue example
from threading import Thread
from queue import PriorityQueue

from time import sleep
from random import randint
from collections import deque

queue = PriorityQueue(10)

for i in range(10):
    value = input("Enter a value: ")
    #queue.put((len(value), value))
    queue.put(value)

for i in range(10):
    v = queue.get()
    print("Value:", v)

Enter a value: 10
Enter a value: 23
Enter a value: 124
Enter a value: 434
Enter a value: 123
Enter a value: 2123
Enter a value: 3213
Enter a value: 123
Enter a value: 123
Enter a value: 123
Value: 10
Value: 123
Value: 123
Value: 123
Value: 123
Value: 124
Value: 2123
Value: 23
Value: 3213
Value: 434


## ThreadPoolExecutor
1. Creates threads lazily as they are submitted but once created it won't remove them.
2. If we are using with ***with clause*** then it will exit only when all the jobs submitted exit

## ProcessPoolExecutor
1. Similar to ThreadPoolExecutor just uses different processes instead. 
2. Use this when you want to perform multiple independant operation using parallel processing. Avoid using it when data is to be share between methods, use ThreadPool instead
3. Each process will get its own copy of global dataw

**concurrent.futures can be used to propagate exception from one thread to another**


In [2]:
#ThreadPoolExecutor example
from concurrent.futures import ThreadPoolExecutor as Executor
#from concurrent.futures import ProcessPoolExecutor as Executor
from time import sleep

def foo():
    print("foo(): invoked...")
    sleep(5)
    return "Hello world - from foo..."

def bar():
    print("bar(): invoked....")
    sleep(60)
    return 100

if __name__ == '__main__':
    with Executor(max_workers=10) as workers:
        f1 = workers.submit(foo)
        f2 = workers.submit(bar)
        print(f1, f2)
        print(f1.done(), f2.done())
        
    print(f1.result(), f2.result())
    print(f1.done(), f2.done())

foo(): invoked...
bar(): invoked....<Future at 0x24fa5986ba8 state=running>
 <Future at 0x24fa5986eb8 state=running>
False False
Hello world - from foo... 100
True True


***workers.map(square, listofargs)***

Creates thread/process and apply worker on each element of the listofargs.


In [3]:
# workers.map example
from concurrent.futures import ThreadPoolExecutor as Executor
from time import sleep, time

def square(x):
    #if x > 10:
    #    raise ValueError("large value...")
    sleep(0.5)
    return x*x

data = [10, 2, 4, 67, 3, 5]


s = time()
with Executor(max_workers=10) as workers:
    result = list(workers.map(square, data))
#result = list(map(square, data))
d = time() - s
print("map took", d, "seconds")


map took 0.5079009532928467 seconds


## Process vs Thread
1. Threads: Global datastructure are visible to all threads
2. Process: Global datastructure changed in one process is not visible in another process. Each process has its own copy of data (copy on write)

In [4]:
#Process vs Thread: Thread Example
from threading import Thread, current_thread
#from multiprocessing import Process as Thread, current_process as current_thread
from time import sleep

class Value:
    def __init__(self, ctype, value):
        self.value = value
    
def foo(val):
    t = current_thread()
    print(f"In foo[{t.name}]: value = {val.value}")
    sleep(2)
    val.value = 1234
    print(f"In foo[{t.name}]: value is set to {val.value}")
    
def bar(val):
    t = current_thread()
    print(f"In bar[{t.name}]: value = {val.value}")
    sleep(3)
    print(f"In bar[{t.name}]: value = {val.value}")
    
if __name__ == '__main__':
    value = Value("i", 100)
    
    t1 = Thread(target=foo, args=(value,))
    t2 = Thread(target=bar, args=(value,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print("In main: value =", value.value)

In foo[Thread-6]: value = 100
In bar[Thread-7]: value = 100
In foo[Thread-6]: value is set to 1234
In bar[Thread-7]: value = 1234
In main: value = 1234


In [5]:
#Process vs Thread: Process Example
#from threading import Thread, current_thread
from multiprocessing import Process as Thread, current_process as current_thread
from time import sleep

class Value:
    def __init__(self, ctype, value):
        self.value = value
    
def foo(val):
    t = current_thread()
    print(f"In foo[{t.name}]: value = {val.value}")
    sleep(2)
    val.value = 1234
    print(f"In foo[{t.name}]: value is set to {val.value}")
    
def bar(val):
    t = current_thread()
    print(f"In bar[{t.name}]: value = {val.value}")
    sleep(3)
    print(f"In bar[{t.name}]: value = {val.value}")
    
if __name__ == '__main__':
    value = Value("i", 100)
    
    t1 = Thread(target=foo, args=(value,))
    t2 = Thread(target=bar, args=(value,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print("In main: value =", value.value)

In main: value = 100


## Data sharing between process: Value
- Use Value class in multiprocessing module to share primitive datatype objects. It should be passed in constructor if it has to be shared between processes

3.
4. For sync & queues use same sync mechanisms and queues from multiprocessing module NOT threading module

In [ ]:
# value example
from multiprocessing import Process, Value, current_process
from time import sleep

def foo(val):
    t = current_process()
    print(f"In foo[{t.name}]: value = {val.value}")
    sleep(2)
    val.value = 1234
    print(f"In foo[{t.name}]: value is set to {val.value}")
    
def bar(val):
    t = current_process()
    print(f"In bar[{t.name}]: value = {val.value}")
    sleep(3)
    print(f"In bar[{t.name}]: value = {val.value}")
    
if __name__ == '__main__':
    # here 'i' is the ctype argument telling Value class of which type the argument is
    value = Value("i", 100)
    # value has to be passed as an argument
    t1 = Process(target=foo, args=(value,))
    t2 = Process(target=bar, args=(value,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print("In main: value =", value.value)


## Data sharing between process: Buffer
- to share a buffer(contiguous memory chunk of same type) use Array from multiprocessing module

In [ ]:
# Buffer example
from multiprocessing import Process, Array, current_process
from time import sleep


def foo(arr):
    t = current_process()
    print(f"In foo[{t.name}]: array = {bytes(arr)}")
    sleep(2)
    for i, v in enumerate(arr):
        arr[i] = v - 32
  
    
def bar(arr):
    t = current_process()
    print(f"In bar[{t.name}]: value = {bytes(arr)}")
    sleep(3)
    for i, v in enumerate(arr):
        if v == 32:
            arr[i] = b'\n'
            
if __name__ == '__main__':
    s = input("Enter a string: ")
    array = Array("i", bytes(s, "utf8"))
    
    t1 = Process(target=foo, args=(array,))
    t2 = Process(target=bar, args=(array,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print("In main: value =", array)
    print("String =", bytes(array))


 ## Data sharing between process: dict or list
 - Sharing dict or list, use multiprocessing import Manager, give managed datastructure, that can be shared between processes

In [ ]:
# Dict/list Example
from multiprocessing import Process, current_process
from multiprocessing import Manager
from time import sleep


def foo(d):
    t = current_process()
    print(f"In foo[{t.name}]: d contains {dict(d)}")
    sleep(2)
    d["name"] = "Smith"
    d["country"] = "India"
  
    
def bar(arr):
    t = current_process()
    sleep(3)
    print(f"In bar[{t.name}]: d  contains {dict(d)}")
    del d["role"]
            
if __name__ == '__main__':
    manager = Manager()
    d = manager.dict()
    
    d["name"] = "Jones"
    d["role"] = "Developer"
    d["place"] = "Hyderabad"
    
    t1 = Process(target=foo, args=(d,))
    t2 = Process(target=bar, args=(d,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print("In main: contents of d =", dict(d))
    print("In main: d =", d)


## Concurrency: asyncio
- built in IO lib. 
- Still undergoing changes
- Not that widely used. gevent is more popular for IO tasks
- More info: https://realpython.com/async-io-python/


In [ ]:
import asyncio
from time import sleep

async def foo():
    for i in range(10):
        print("foo: counting", i)
        await asyncio.sleep(0.5)

async def bar():
    for i in range(10):
        print("bar: counting", i)
        await asyncio.sleep(0.5)

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(asyncio.gather(foo(), bar()))



## Sync usage recommendations
- ** If the work has more than ***50%*** duty cycle recommended to use ***processes***
- ** If the duty cyle is ***70-100%*** then number of processes should not exceed number of cores. multiprocessing.cpu_count()
- ** If the duty cycle is ***0.1 - 30%*** or  below then use ***threads*** i.e. concurrent operation
- ** duty cycle less than use ***gevent***, as they yield CPU before blocking
- ** Offload CPU intensive work to processpool and IO intensive work to eventloop of gevent


## Profiling a function in python


In [ ]:
python -m CProfile <program>


## Profiling a program in Linux

Use time command

Then
**duty cycles = real / (user + sys)**

In [ ]:
time <program_name>